In [ ]:

%run includes/3-network-setup.ipynb
networked_projects = load_object('networked_projects')

# Networks (Hybrid Connectivity)
Summary and detail reports on hybrid connectivity (Interconnects, VPNs, etc) across the GCP estate.

## Interconnects

In [ ]:

# get interconnects
interconnect_queries = [
    f"""
    SELECT *
    FROM google.compute.interconnects WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in networked_projects
]
interconnects_df = run_stackql_queries(interconnect_queries)
interconnects_df


## VPN Gateways

In [ ]:

# get vpns
vpn_queries = [
    f"""
    SELECT
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 9) as network,
    SPLIT_PART(region, '/', 8) as region,
    JSON_ARRAY(JSON_EXTRACT(vpnInterfaces, '$[0].ipAddress'), JSON_EXTRACT(vpnInterfaces, '$[1].ipAddress')) as ip_addresses
    FROM google.compute.vpn_gateways WHERE project = '{project}'
    """
    for project in networked_projects
]
vpns_df = run_stackql_queries(vpn_queries)
vpns_df

## External VPN Gateways

In [ ]:
external_vpn_gateways_queries = [
    f"""
    select 
    name
    ,description
    ,'{project}' as project
    ,JSON_ARRAY_LENGTH(interfaces) as num_interfaces
    ,JSON_ARRAY(JSON_EXTRACT(interfaces, '$[0].ipAddress'), JSON_EXTRACT(interfaces, '$[1].ipAddress'), JSON_EXTRACT(interfaces, '$[2].ipAddress'), JSON_EXTRACT(interfaces, '$[3].ipAddress')) as ip_addresses
    ,redundancyType
    ,labels 
    from google.compute.external_vpn_gateways where project = '{project}'
    and name is not null
    """
    for project in networked_projects
]
external_vpn_gateways_df = run_stackql_queries(external_vpn_gateways_queries)
external_vpn_gateways_df


### VPN Tunnels

In [ ]:

# vpn tunnels
vpn_tunnels_queries = [
    f"""
    select 
    name,
    description,
    '{project}' as project,
    SPLIT_PART(region, '/', 9) as region,
    status,
    detailedStatus,
    SPLIT_PART(router, '/', 11) as router,
    SPLIT_PART(vpnGateway, '/', 11) as vpnGateway,
    targetVpnGateway,
    vpnGatewayInterface,
    peerGcpGateway,
    ikeVersion,
    peerIp,
    peerExternalGatewayInterface,
    SPLIT_PART(peerExternalGateway, '/', 10) as peerExternalGateway,
    localTrafficSelector,
    remoteTrafficSelector,
    labels
    from google.compute.vpn_tunnels where project = '{project}'
    """
    for project in networked_projects
]
vpn_tunnels_df = run_stackql_queries(vpn_tunnels_queries)
vpn_tunnels_df

In [ ]:

# save data
save_object(len(interconnects_df), 'num_interconnects')
save_object(len(vpns_df), 'num_vpns')
save_object(len(external_vpn_gateways_df), 'num_external_vpn_gateways')
save_object(len(vpn_tunnels_df), 'num_vpn_tunnels')